In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Keypress

In [2]:
keyPress = pd.read_csv('../agg_datasets/KeyPressEvent.csv',index_col = 0)

/Users/thicorfon/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
keyPress = keyPress.rename(columns={'PressType':'ActivityID','ActivityID':'PressType'})

In [4]:
keyPress['SessionID'] = np.floor(keyPress.ActivityID/10e6).astype(int)

In [36]:
aux = keyPress[['Systime','SessionID']].groupby('SessionID').agg([np.min,np.max]).Systime

In [46]:
aux['diff'] = aux.amax - aux.amin
aux = aux.reset_index()

In [49]:
bla = []
step = 5000
for _,row in aux.iterrows():
    i = 0
    t0 = row.amin
    while (t0+step < row.amax):
        aux_dict = {}
        aux_dict['SessionID'] = row.SessionID
        aux_dict['WindowNumber'] = i
        aux_dict['WindowStart'] = t0
        aux_dict['WindowEnd'] = t0+step
        t0 += step
        i += 1
        bla.append(aux_dict)

In [52]:
fim = pd.DataFrame(bla)

In [56]:
fim.to_csv('skeleton.csv',index=False)

In [5]:
fim = pd.read_csv('skeleton.csv')

In [11]:
fim[(fim.SessionID == 10066918) & (fim.WindowStart < 1396571749367) & (fim.WindowEnd > 1396571749367)].WindowNumber.values[0]

1

In [12]:
keyPress

,Systime,PressTime,ActivityID,PressType,KeyID,Phone_orientation,SessionID
0,1396571743474,5179588,100669181000001,1,-1,0,10066918
1,1396571743698,5179506,100669181000001,0,-1,0,10066918
2,1396571744401,5180514,100669181000001,1,109,0,10066918
3,1396571744646,5180453,100669181000001,0,109,0,10066918
4,1396571744721,5180835,100669181000001,1,121,0,10066918
5,1396571744975,5180782,100669181000001,0,121,0,10066918
6,1396571746135,5182245,100669181000001,1,32,0,10066918
7,1396571746393,5182196,100669181000001,0,32,0,10066918
8,1396571746678,5182796,100669181000001,1,102,0,10066918
9,1396571746924,5182732,100669181000001,0,102,0,10066918


In [13]:
%%time
aux = keyPress.merge(fim,on='SessionID')

CPU times: user 22.3 s, sys: 52.7 s, total: 1min 14s
Wall time: 2min 5s


In [ ]:
aux = aux[(aux.Systime >= aux.WindowStart) & (aux.Systime <= aux.WindowEnd)]

In [12]:
def discoverWindowNumber(row,skeleton=fim):
    x = skeleton[(skeleton.SessionID == row.SessionID) & (skeleton.WindowStart < row.Systime) & (fim.WindowEnd > row.Systime)]
    if x.empty:
        return None
    else:
        return x.WindowNumber.values[0]

In [16]:
i = 0
for _,row in keyPress.iterrows():
    i += 1
    if i == 100:
        l = row
        break

In [20]:
keyPress['WindowNumber'] = keyPress.apply(discoverWindowNumber,axis=1)

In [23]:
keyPress.dropna().to_csv('../KeyPressEvent.csv',index=False)

# Magnetometer

In [4]:
keyPress = pd.read_csv('../agg_datasets/TouchEvent.csv')

/Users/thicorfon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
keyPress

,Systime,EventTime,ActivityID,Pointer_count,PointerID,ActionID,X,Y,Pressure,Contact_size,Phone_orientation,SessionID
0,1399330689901,919805.0,3.423291e+14,1.0,0.0,0.0,1012.00000,130.000000,1.0,0.023529,0.0,34232910.0
1,1399330689935,919837.0,3.423291e+14,1.0,0.0,2.0,1012.00000,130.000000,1.0,0.023529,0.0,34232910.0
2,1399330689953,919849.0,3.423291e+14,1.0,0.0,2.0,1012.00000,130.000000,1.0,0.023529,0.0,34232910.0
3,1399330689966,919871.0,3.423291e+14,1.0,0.0,2.0,1012.00000,130.000000,1.0,0.025490,0.0,34232910.0
4,1399330689982,919887.0,3.423291e+14,1.0,0.0,2.0,1012.00000,130.000000,1.0,0.025490,0.0,34232910.0
5,1399330689999,919904.0,3.423291e+14,1.0,0.0,2.0,1012.00000,130.000000,1.0,0.023529,0.0,34232910.0
6,1399330690014,919920.0,3.423291e+14,1.0,0.0,2.0,1012.00000,130.000000,1.0,0.023529,0.0,34232910.0
7,1399330690020,919929.0,3.423291e+14,1.0,0.0,1.0,1012.00000,130.000000,1.0,0.023529,0.0,34232910.0
8,1399330690325,920231.0,3.423291e+14,1.0,0.0,0.0,266.00000,78.000000,1.0,0.025490,0.0,34232910.0
9,1399330690352,920253.0,3.423291e+14,1.0,0.0,2.0,266.00000,78.000000,1.0,0.027451,0.0,34232910.0


In [13]:
keyPress['SessionID'] = np.floor(keyPress.ActivityID/10e6)

In [14]:
skeleton = pd.read_csv('skeleton.csv')

In [15]:
skeleton

,SessionID,WindowEnd,WindowNumber,WindowStart
0,10066902,1396227055527,0,1396227050527
1,10066902,1396227060527,1,1396227055527
2,10066902,1396227065527,2,1396227060527
3,10066902,1396227070527,3,1396227065527
4,10066902,1396227075527,4,1396227070527
5,10066902,1396227080527,5,1396227075527
6,10066902,1396227085527,6,1396227080527
7,10066902,1396227090527,7,1396227085527
8,10066902,1396227095527,8,1396227090527
9,10066902,1396227100527,9,1396227095527


In [17]:
aux_list = []
last_session = -1
last_windowNumber = -1
last_windowEnd = -1
count = 0
j = 0
for _,row in keyPress.iterrows():
    if row.SessionID != last_session:
        skeletonWithSessionId = skeleton[skeleton.SessionID == row.SessionID]
        last_session = row.SessionID
    currentSystime = int(row.Systime)
    if currentSystime > last_windowEnd:
        final_df = skeletonWithSessionId[(skeletonWithSessionId.WindowStart <= currentSystime) & (skeletonWithSessionId.WindowEnd > currentSystime)]
        if final_df.empty:
            last_windowEnd = -1
            last_windowNumber = -1
        else:
            last_windowEnd = final_df.WindowEnd.values[0]
            last_windowNumber = final_df.WindowNumber.values[0]
    aux_list.append(last_windowNumber)
    count = count + 1
    j = j + 1
    if (j == 100000):
        print(count)
        j = 0
keyPress['WindowNumber'] = aux_list

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000


In [ ]:
keyPress.to_csv('../windowed/TouchEvent.csv',index=False)